In [1]:
import numpy as np

In [2]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=2#检测零件1的成本
    C2=3#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=6#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update,n_update

In [3]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
#第一阶段的生产决策
result=[]
for x1 in [0,1]:
    for x2 in [0,1]:
        for y in [0,1]:
            for z in [0,1]:
                profit,n_update,p1_update,p2_update,n_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update,n_update])
#第二阶段的决策
result1=np.empty((8, 16), dtype=object)
for i in range(1,16,2):
    count=0
    for x1 in [0,1]:
        for x2 in [0,1]:
            for y in [0,1]:
                for z in [0,1]:
                    profit,n_update,p1_update,p2_update,n_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                    result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update,n_update]
                    count+=1
#第三阶段决策
result2=np.empty((8,8,16), dtype=object)
for i in range(8):
    for j in range(1,16,2):
        count=0
        for x1 in [0,1]:
            for x2 in [0,1]:
                for y in [0,1]:
                    for z in [0,1]:
                        profit,n_update,p1_update,p2_update,n_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                        result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update,n_update]
                        count+=1

#第四阶段决策
result3=np.empty((8,8,8,16), dtype=object)
for i in range(8):
    for j in range(8):
        for k in range(1,16,2):
            count=0
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update,n_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                            result3[i,j,int((k-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update,n_update]
                            count+=1

In [4]:
result3_value=np.zeros((8,8,8,16))
for i in range(8):
    for j in range(8):
        for k in range(8):
            for g in range(16):
                result3_value[i,j,k,g]=result3[i,j,k,g][4]
result3_value_max=np.max(result3_value,axis=3)

In [5]:
result2_value=np.zeros((8,8,16))
for i in range(8):
    for j in range(8):
        for k in range(16):
            result2_value[i,j,k]=result2[i,j,k][4]
result2_value[:,:,1::2]=result2_value[:,:,1::2]+result3_value_max
result2_value_max=np.max(result2_value,axis=2)

In [6]:
result1_value=np.zeros((8,16))
for i in range(8):
    for j in range(16):
        result1_value[i,j]=result1[i,j][4]
result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
result1_value_max=np.max(result1_value,axis=1)

In [7]:
final_result=[0 for i in range(16)]
for i in range(16):
    final_result[i]=result[i][4]
final_result[1::2]=final_result[1::2]+result1_value_max
max_value=max(final_result)
max_index = final_result.index(max_value)
print(f"最终的最大利润为{max_value}")
print("第一阶段的生产决策为:")
print(result[max_index])
if max_index%2:
    max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
    print("第二阶段的生产决策为:")
    print(result1[int((max_index-1)/2),max_index_1])
if max_index_1%2:
    max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
    print("第三阶段的生产决策为:")
    print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
if max_index_2%2:
    max_index_3 = np.argmax(result3_value[int((max_index-1)/2),int((max_index_1-1)/2),int((max_index_2-1)/2)])
    print("第四阶段的生产决策为:")
    print(result3[int((max_index-1)/2),int((max_index_1-1)/2),int((max_index_2-1)/2),max_index_3])

最终的最大利润为451146.99
第一阶段的生产决策为:
[0, 0, 0, 1, 318430.0, 2710.0, 0.1, 0.1, 2710.0]
第二阶段的生产决策为:
[0, 0, 1, 1, 82570.98999999999, 734.0, 0.1, 0.1, 734.0]
第三阶段的生产决策为:
[1, 1, 0, 1, 24908.0, 66.0, 0.0, 0.0, 66.0]
第四阶段的生产决策为:
[1, 1, 0, 0, 25238.0, 0, 0.0, 0.0, 0]
